# Selenium

Objetivo: explorar a biblioteca Selenium para acessar o site da CVM e extrair o relatório financeiro Formulário de Referência.

In [2]:
from random import randint
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

from time import sleep

In [3]:
print(webdriver.__version__)

4.10.0


In [4]:
# Iniciando o navegador.
driver = webdriver.Chrome()

# Acessar o site da CVM.
link = 'https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx'
driver.get(link)
assert driver.title == 'Consulta de Documentos de Companhias Abertas'

# Simula uma pausa.
#wait_time = randint(3, 6)
#driver.implicitly_wait(wait_time)
driver.implicitly_wait(10)

In [5]:
# Escolher o relatório Formulário de Referência.
text_box_documento = driver.find_element(By.ID, 'cboCategorias_chosen_input')
driver.implicitly_wait(5)

text_box_documento.send_keys('FRE - Formulário de Referência')
driver.implicitly_wait(5)
text_box_documento.send_keys('\ue007')

In [6]:
# Clicar no botão 'No período' para habilitar a seleção da data.
radio_button_periodo = driver.find_element(By.ID, 'rdPeriodo')
radio_button_periodo.click()
driver.implicitly_wait(5)

# Selecionar a data de início
text_box_dt_inicio = driver.find_element(By.ID, 'txtDataIni')
#text_box_dt_inicio.click()
driver.implicitly_wait(5)
text_box_dt_inicio.send_keys('01012021')

# Selecionar a data de início
text_box_dt_inicio = driver.find_element(By.ID, 'txtDataFim')
#text_box_dt_inicio.click()
driver.implicitly_wait(5)
text_box_dt_inicio.send_keys('31012021')

In [7]:
# Selecionar a empresa (opcional)
#text_box_empresa = driver.find_element(By.ID, 'cboEmpresa')
#text_box_empresa.send_keys('Porto Seguro S.A.')
#driver.implicitly_wait(3)
#text_box_empresa.send_keys(Keys.DOWN)
#driver.implicitly_wait(5)
#text_box_empresa.send_keys('\ue007')
#text_box_empresa.send_keys(Keys.RETURN)

# não funcionou.

In [8]:
# Apertar botão de execução da consulta.
button_consulta = driver.find_element(By.ID, 'btnConsulta')
button_consulta.click()

In [9]:
# Listar todos os downloads possíveis.
list_docs = driver.find_elements(By.ID, 'VisualizarDocumento')
print(list_docs[:3])
print(list_docs[0].get_attribute('onclick'))

[<selenium.webdriver.remote.webelement.WebElement (session="6aa790ab87bbecb76bbe8224b4071b62", element="F4477AD965E118571D96134CFAF4C460_element_119")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa790ab87bbecb76bbe8224b4071b62", element="F4477AD965E118571D96134CFAF4C460_element_120")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa790ab87bbecb76bbe8224b4071b62", element="F4477AD965E118571D96134CFAF4C460_element_121")>]
OpenPopUpVer('frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99816&CodigoTipoInstituicao=1')


In [18]:
# Experimentando como extrair o relatório da página.

for i in list_docs:
        
    url_suffix = i.get_attribute('onclick')
    url_suffix = url_suffix[
        url_suffix.find("'") + 1:
        -2
    ]

    single_link = f"https://www.rad.cvm.gov.br/ENET/{url_suffix}"
    print(single_link)

    single_driver = webdriver.Chrome()
    single_driver.get(single_link)

    # Clique no botão 'Salvar em PDF'.
    wait = WebDriverWait(single_driver, timeout=5)
    wait.until(expected_conditions.element_to_be_clickable((By.ID, "btnGeraRelatorioPDF"))).click()
    #salvar_em_pdf = single_driver.find_element(By.ID, 'btnGeraRelatorioPDF')
    #print(single_driver.page_source)

    # Antes de gerar o PDF, é necessário entrar no iframe (i.e. html externo dentro html principal).
    iframe = single_driver.find_element(By.CSS_SELECTOR, '#iFrameModal')
    single_driver.switch_to.frame(iframe)

    #Clique no botão 'Gerar PDF'
    wait = WebDriverWait(single_driver, timeout=5)

    sleep(5)

    wait.until(expected_conditions.element_to_be_clickable((By.TAG_NAME, 'button'))).click()

    sleep(20)
    single_driver.quit()

#salvar_em_pdf.click()
# Método mais complexo para clicar
#ActionChains(single_driver).click(salvar_em_pdf).perform()

#print('--------------------------------')

#single_driver.implicitly_wait(10)
#print(single_driver.page_source)
#wait = WebDriverWait(driver, 10)
#element = wait.until(expected_conditions.presence_of_element_located((By.ID, 'btnConsulta')))

#wait = WebDriverWait(driver, 10)
#new_div = wait.until(expected_conditions.visibility_of_element_located((By.ID, 'btnConsulta')))
#second_button = new_div.find_element_by_id('btnConsulta').click()


#updated_html = single_driver.page_source
#updated_driver = webdriver.Chrome()
#element = single_driver.switch_to.active_element

#gerar_pdf = single_driver.find_element_by_id('btnConsulta')
#gerar_pdf.click()

https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99816&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99815&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99812&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99807&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99803&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99799&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99034&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=99778&CodigoTipoInstituicao=1
https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=9977

KeyboardInterrupt: 

In [169]:
#updated_driver.find_element(By.ID, 'btnConsulta')
#single_driver.current_url
#updated_driver = webdriver.Chrome()
#updated_driver.get(single_link)
#updated_driver.find_element(By.ID, 'btnConsulta')
#print(single_driver.page_source)
#element.find_element(By.ID, 'btnConsulta')
#iframe = single_driver.find_element(By.ID, "iFrameModal")
#teste = single_driver.switch_to.frame(iframe)
#teste
#salvar_em_pdf.element_to_be_clickable()

# Check: o objetvo é clicável? 
#from selenium.common.exceptions import WebDriverException
#try:
#    salvar_em_pdf.click()
#except WebDriverException:
#    print("Element is not clickable")

#iframe = single_driver.find_element(By.CSS_SELECTOR, '#iFrameModal')
#single_driver.switch_to.frame(iframe)


In [170]:
#print(single_driver.page_source)
#iframe.get_attribute('id')


In [25]:
# Aperta o botão de próxima página.
print('pag1')
button_proximo = driver.find_element(By.ID, 'grdDocumentos_next')
button_proximo.click()
driver.implicitly_wait(5)

print('pag2')
button_proximo = driver.find_element(By.ID, 'grdDocumentos_next')
button_proximo.click()
driver.implicitly_wait(5)

#print('pag3')
#button_proximo = driver.find_element(By.ID, 'grdDocumentos_next')
#button_proximo.click()
#driver.implicitly_wait(5)

pag1
pag2
